# Doc2Vec Model Training

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random
import ast
from nltk.corpus import stopwords
from gensim.utils import tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
df = pd.read_csv('../data/lyrics.csv')

In [3]:
df.head()

,lyrics,genre,outlier,id
0,"['Go, go, go, go', 'Go, go, go shawty', ""It's ...",rap,False,0
1,"['New York City!', 'You are now rapping...with...",rap,False,1
2,"[""I don't know what you heard about me"", ""But ...",rap,False,2
3,"['Man we gotta go get something to eat man', ""...",rap,False,3
4,"['Yeah...', 'Uh huh', 'So seductive', ""I'll ta...",rap,False,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37015 entries, 0 to 37014
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lyrics   37015 non-null  object
 1   genre    37015 non-null  object
 2   outlier  37015 non-null  bool  
 3   id       37015 non-null  int64 
dtypes: bool(1), int64(1), object(2)
memory usage: 903.8+ KB


In [5]:
df['lyrics'] = list(map(lambda x: ast.literal_eval(x), df['lyrics']))

In [6]:
idxs_eval = pd.read_csv('../data/evaluation/lyrics_eval.csv')['id']
idxs_test = pd.read_csv('../data/classification/lyrics_test.csv')['id']

In [7]:
indexes2delete = list(idxs_eval) + list(idxs_test)

In [8]:
df.drop(indexes2delete, inplace=True)
df.reset_index(inplace=True)

In [9]:
verses = [verse for lyrics in df['lyrics'] for verse in lyrics]

In [10]:
#lyrics_with_genre = zip(list(df['lyrics']), list(df['genre']))

#verses_with_genre = [(verse, genre) for lyrics, genre in lyrics_with_genre for verse in lyrics]

#verses = [verse for verse, genre in verses_with_genre]
#genres = [genre for verse, genre in verses_with_genre]

## Doc2Vec

### Tokenization

In [11]:
tokenized_verses = [list(tokenize(verse, lowercase=True)) for verse in tqdm(verses)]

100%|██████████| 1509821/1509821 [00:24<00:00, 62604.18it/s] 


In [12]:
stop_words = stopwords.words('english')
tokenized_verses = list(map(lambda x: [token for token in x if token not in stop_words], tokenized_verses))
tokenized_verses = [verse for verse in tokenized_verses if verse]
#tokenized_verses_with_genre = [(verse, genre) for verse, genre in zip(tokenized_verses, genres) if verse]

In [13]:
#Xverses = [verse for verse, genre in tokenized_verses_with_genre]
#y = [genre for verse, genre in tokenized_verses_with_genre]

In [14]:
#Xverses_train, Xverses_val, y_train, y_val = train_test_split(Xverses, y, test_size=0.05, random_state=0, stratify=y)

## Doc2Vec

In [15]:
documents = [TaggedDocument(verse, [i]) for i, verse in tqdm(enumerate(tokenized_verses))]

1494297it [00:07, 189764.38it/s]


In [16]:
model = Doc2Vec(min_count=2, epochs=20, dbow_words=1, vector_size=50)

model.build_vocab(documents)

model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

model.save('../models/doc2vec.model')
